<a href="https://colab.research.google.com/github/tjubes/DataMining/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [204]:
import numpy as np
import collections
import pandas as pd
import random
from collections import Counter
from scipy import stats

class DecisionNode():
    def __init__(self, feature=None, splitpoint=None, left=None, right=None, value=None, leaf=False):
      self.feature = feature          # Feature to split on, column index
      self.splitpoint = splitpoint    # Threshold value for the split
      self.left = left                # Left subtree
      self.right = right              # Right subtree
      self.value = value              # If leaf node, the predicted class
      self.leaf = leaf                # If node is leaf node

    def is_leaf_node(self):
      return self.leaf

def impurity(a):
  i = (np.count_nonzero(a == 0)/len(a))*(1-np.count_nonzero(a == 0)/len(a))
  return i

def bestsplit(x,y, minleaf):
# this function is for a single feature
#return best splitpoint, reduction of splitpoint (gini index)

  #The best split is the split that achieves the highest impurity reduction.
  impurity_parent = impurity(y)

  x_sorted = np.sort(np.unique(x))
  x_splitpoints = (x_sorted[0:(len(x_sorted)-1)]+x_sorted[1:(len(x_sorted))])/2

  reductions = []
  for s in x_splitpoints:
    #go over all the splitpoints
    left = impurity(y[x <= s])
    l = len(y[x <= s])/len(y)
    right = impurity(y[x > s])
    r = len(y[x > s])/len(y)

    #calculate gini reduction
    reduction = impurity_parent - ((l*left) + r*(right))

    #only if minleaf parameter is correct
    if (len(y[x<=s]) >= minleaf) and (len(y[x>s]) >= minleaf):
      reductions.append(reduction)

  # the reductions list is empty if the minleaf constraint is not satisfied
  if reductions == []:
    return None, 0

  best_splitpoint = x_splitpoints[reductions.index(max(reductions))]

  return best_splitpoint, max(reductions)

def split(x,y,minleaf,nfeat):
  #this function finds which feature has the best split, using the best_split function
  # which feature do we split on, and what is the threshold?
  best_reduction = 0
  best_splitpoint = None
  column_i = None
  i = 0
  n_features = len(x.T)

  #which random features we split on
  random_features = random.sample(range(0, n_features), nfeat)

  for index in random_features:
    splitpoint, reduction = bestsplit(x[:, index],y,minleaf)

    if reduction > best_reduction:
      best_reduction = reduction
      best_splitpoint = splitpoint
      column_i = index
    i+=1
  return best_splitpoint, column_i

def tree_grow(x, y, nmin, minleaf, nfeat):
  observations = len(x)

  #the number of observations that a node must contain at least, for it to be
  #allowed to be split
  #also if the node is already pure, splitting is not needed anymore
  if observations < nmin or np.all(y == y[0]): #is np.all(y == 0) correct?
    return DecisionNode(value=y, leaf=True)

  #find the column that has the best quality of split
  splitpoint, feature_index = split(x,y,minleaf,nfeat)

  #if the minleaf  constraint is not sasisfied, so if no split can be found that
  #creates a node with fewer than minleaf observations is not acceptable.
  #leaf node created
  if feature_index == None:
    return DecisionNode(value=y, leaf=True)

  #left
  y_l = y[x[:,feature_index]<=splitpoint]
  x_l = x[np.where(x[:,feature_index]<=splitpoint)]

  #right
  y_r = y[x[:,feature_index]>splitpoint]
  x_r = x[np.where(x[:,feature_index]>splitpoint)]

  #recursion
  left_tree = tree_grow(x_l,y_l, nmin, minleaf, nfeat)
  right_tree = tree_grow(x_r,y_r, nmin, minleaf, nfeat)

  return DecisionNode(feature=feature_index, splitpoint=splitpoint, left=left_tree, right=right_tree, value=y)

In [175]:
#data
df = pd.read_csv('pima.txt', header=None).to_numpy()

x = df[:, 0:8]
y = df[:, 8]

x_test = df[0:3, 0:8]
y_test = df[0, 8]

In [172]:
tr = tree_grow(x,y,10,10,8)

In [177]:
x_test

array([[  6.   , 148.   ,  72.   ,  35.   ,   0.   ,  33.6  ,   0.627,
         50.   ],
       [  1.   ,  85.   ,  66.   ,  29.   ,   0.   ,  26.6  ,   0.351,
         31.   ],
       [  8.   , 183.   ,  64.   ,   0.   ,   0.   ,  23.3  ,   0.672,
         32.   ]])

In [189]:
def tree_pred_one(x, tr):
    # returns labels

    # if a leaf node is found
    if tr.is_leaf_node() == True:
        counter = Counter(tr.value)
        majority_value = counter.most_common(1)[0][0]
        return majority_value
    
    # otherwise see whether we should go left or right in the decision tree
    split_feature = tr.feature
    splitpoint = tr.splitpoint

    if x[split_feature] > splitpoint:
        return tree_pred_one(x,tr.right)
    if x[split_feature] <= splitpoint:
        return tree_pred_one(x,tr.left)

# Tree prediction with just 1 tree, so without bagging
def tree_pred(x, tr):
    predictions = []
    
    for datapoint in x:
        predictions.append(tree_pred_one(datapoint, tr))
    
    return np.array(predictions)


In [192]:
print(tree_pred(x_test,tr))

[1. 0. 1.]


In [196]:
# Bagging tree grow, i.e. tree grow with bootstrap aggregating
def tree_grow_b(x, y, nmin, minleaf, nfeat, m):
    tree_bags = []
    for i in range(m):
        tree = tree_grow(x,y,nmin,minleaf,nfeat)
        tree_bags.append(tree)

    return tree_bags # should be a list of trees

In [211]:
# Tree prediction with bagging
def tree_pred_b(x, tree_bags):
    # for each store what the trees would predict
    predictions = []
    for tree in tree_bags:
        predictions.append(tree_pred(x, tree))

    #find the majority prediction
    Y = stats.mode(np.array(predictions).T, axis=1, keepdims=False)

    return Y.mode.flatten() # Should return a vector y where y[i] contains the predicted class label for row i of x


In [212]:
print(tree_pred_b(x_test, tree_grow_b(x, y, 10, 10, 4, 6)))

[1. 0. 1.]
